# Abstract

# Introduction

This blog post explores how features of evolution in nature can inspire solutions to overcoming the shortcomings of gradient descent. Gradient Descent only works on differentiable loss functions, meaning it can become stuck in local loss minima when attempting to model non-convex loss functions. In other words, gradient descent cannot explore the entire solution space on nondifferentiable loss functions. This limitation can be overcome by harnessing the characteristics of evolution and natural selection in nature. Evolution has a wide variety of applications concerning Machine Learning, but this project focuses on its applications to weight vector optimization @Talikani2021evolutionary.


Lewontin identifies 3 key population characteristics for evolution: phenotypic variation in a population, differential fitness, and fitness must be heritable @Lewontin1970units. With these 3 characteristics, evolution then occurs as ‘fitter’ individuals are better able to pass on their traits to future generations, while less fit individuals are not. At the individual level, evolution requires a blueprint, self-replication, mutation, and selection. By applying these principles to machine learning models, this blog post explores the strengths and limitations of evolutionary principles when applied to weight vector optimization in machine learning. To satisfy the requirement of phenotypic variation, each evolutionary optimizer has an entire population of weight vectors storing different weights. The different weights result in different losses, which in combination with selection pressures regarding the resulting different losses, satisfy the differential fitness requirement. With weight vectors serving as our genetic blueprint, those weight vectors can be duplicated to create or refill the population of weight vectors. Slight random adjustments to those weight vectors during replication serve as the mutations, ensuring the continuation of phenotypic variation. A variety of methods can be used to eliminate population vectors during an iteration, including loss and diversity, which function as selection. Eliminating high-loss weight vectors allows only vectors with high accuracy to pass on their characteristics, while eliminating low diversity can ensure that the solution space is adequately explored. Through the implementation of hyperparameters, many variations of evolutionary machine learning algorithms are explored to better understand their strengths and weaknesses.


The many hyperparameters are then tested on both generated and real data from the MNIST dataset to develop initial hypotheses regarding the optimal parameterization for evolutionary weight vector optimization to succeed. 


# Values Statement

The potential users of the evolutionary-based weight vector optimization class are researchers, data scientists, and developers, especially those who work on non-differentiable problems with which gradient descent-based solutions struggle. Our class provides both a potential solution to overcoming the limitations of gradient descent on non-differentiable classification problems and serves as a potential benchmark against which other algorithms can be compared. 


One major potential impact of the widespread use of our algorithm, or similar ones, is the increase in computational power required to run them. Because each epoch of an evolutionary algorithm requires the computation of an entire population of new weight vectors, the computational power required for an epoch is higher than most algorithms. This has potential positive implications for the manufacturers of computational chips and the owners of servers. On the other hand, the potential negative effects of increased energy and material consumption to perform these computations cannot be overlooked either. 


Because the majority of our work was focused on the creation of a class, and not the optimization of a specific algorithm, the potential for positive and negative impacts of our class depends on who gains access to the class and what they decide to do with it. 


# Materials and Methods

Project dependencies:
- `torch`
- `numpy`
- `pandas`
- `scikit-learn`
- `matplotlib`

Our evolutionary optimizer translates biological principles into a deep neural network optimiser. Biological evolution, and thus our algorithmic approach, rely on four core attributes: blueprint, self-replication, mutation, and selection. Our “blueprints,” genes or DNA in the natural world, are our weight vectors for the parameters of the neural network. We begin with an initial “population” of $N$ such vectors that are sampled uniformly at random. In each generation, every individual is evaluated on a mini-batch of examples, combining cross-entropy loss (exploitation) with an optional diversity penalty (exploration). The lowest‐loss individuals (and occasionally a small “sneaker” fraction of high-loss outliers) serve as parents for the next generation. Some elite low-loss survivors carry forward unchanged. New offspring are created via uniform crossover, where each weight entry, or gene, is inherited from $k$ randomly chosen parents, then mutated by adding small Gaussian or Laplacian noise with some probability. Optionally, each child can receive a single gradient‐descent step to fine-tune its accuracy.
Initially, we relied on synthetic binary-classification data generated using `torch.rand` to train and validate our evolutionary approach. This allowed us to develop a proof of concept that evolution could, in fact, solve problems and that our selection, crossover, mutation, etc., behaved as expected before we moved on to real-world inputs.

<img src="images/fitness/mnist_ex.png" width="400" style="display:block; margin:auto;" />

<p style="text-align: center;"><em>Example of MNIST Digits</em></p>

We decided to employ our evolutionary approach to the MNIST handwritten-digit dataset @lecun2010mnist.  This dataset is made up of 70,000 gray-scale images of size 28×28 pixels, labeled 0–9. We accessed the dataset through `torch` datasets. In smaller experiments with the MNIST dataset, we opted to draw a random subset of anywhere from 1,000 to 20,000 digits to improve computational efficiency. Although the smaller subsets enabled rapid prototyping, they may have overrepresented certain rarer handwriting styles and potentially skewed accuracy.


# Hyperparameters

### Proof of concept/vanilla evolution:

### Selection Processes:

In nature, genetic traits are passed from one generation to the next by individuals that survive and successfully reproduce. These survivors make up the gene pool of their generation, while those that fail to reproduce are effectively excluded from the evolutionary process. In our implementation, we emulate this principle by defining fitness based on standard cross-entropy loss or diversity-augmented loss, depending on the user. At each generation, we sort the population in a minheap based on loss, then the top 50% (the half with the lowest loss) are selected to form the gene pool. The other half does not have the chance to reproduce. In the next section, we will dive into how we handle creating the next generation from the gene pool.

<img src="images/fitness/basic.png" width="750" style="display:block; margin:auto;" />

<p style="text-align: center;"><em>Illustration of Gene Pool Selection</em></p>

Mirroring the random nature of evolution, we incorporate some chance in the makeup of our gene pool. A small number of lower-performing individuals (10% by default) are included in the gene pool with low probability. These individuals, whom we call sneakers, introduce genetic variation that helps maintain a diversified population and prevents premature convergence.

<img src="images/fitness/sneaker.png" width="750" style="display:block; margin:auto;" />

<p style="text-align: center;"><em>Illustration of Sneaker Population</em></p>

Finally, we employ an elitist strategy to preserve our high-performing solutions. Each generation, a percentage of the top performers based purely on cross-entropy loss are included in the gene pool and also survive unchanged and unmutated to the next generation. This preserves the integrity of the best solutions by keeping a lineage of high-performing individuals.

<img src="images/fitness/final.png" width="750" style="display:block; margin:auto;" />

<p style="text-align: center;"><em>Overview of New Generation Gene Makeup</em></p>

### Inheritance and Parent Quantity:


At each iteration of our evolutionary optimization, following the creation of a ‘gene pool’ in the selection stage, the population must be replenished with new individuals. There are three ways that this can be accomplished. 1: All new individuals are new randomized weight vectors with no input from the gene pool. 2: Each new individual has a single parent randomly selected from the gene pool from which its weights are inherited with random mutations. 3: Each individual has n parents randomly selected from the gene pool. Each feature weight is then inherited from the corresponding feature weight of a random one of its parents. 

The first scenario, with no inherited weight vectors, is a baseline against which our true evolutionary models can be tested. This is not truly evolution, as it does not include any heritability of fitness for new individuals in the population @Lewontin1970units.

The second Scenario, includes heritability of fitness, but with only a single parent for each child individual, the diversity can be expected to be more limited.

<img src="images/Multi-Parent/Single_Parent_Inheritance.png" width="400" style="display:block; margin:auto;" />

<p style="text-align: center;"><em>Diagram of Inheritance when num_parents = 1</em></p>



The Third Scenario, allows for a slightly reduced heritability of fitness, with the addition of diverse new individuals produced with each generation. 

<img src="images/Multi-Parent/Multi-parent_Inheritance.png" width="400" style="display:block; margin:auto;" />

<p style="text-align: center;"><em>Diagram of Inheritance when num_parents = 2</em></p>


As discussed in the results section, the choice of the number of parents can have a significant impact on loss, accuracy, and diversity.

### Hybridizing evolution with gradient descent:

### Adjustment from binary to multi-class classification:

### Mutation Methods:

# Results

### Proof of concept/vanilla evolution:



### Choices in Selection Processes:

By default, we select the best 50% of the population to enter the gene pool, however, this is a hyperparameter that users can play with. We conducted some experiments on a small subset of 1000 digits from the MNIST dataset to examine how different gene pool sizes (10%–90% of the population) would affect our accuracy, loss, and diversity over 500 generations. 

<img src="images/fitness/Pool vs Acc.png" width="750" style="display:block; margin:auto;" />
<p style="text-align: center;"><em>Gene Pool Size vs Accuracy</em></p>

<img src="images/fitness/Pool vs Loss.png" width="750" style="display:block; margin:auto;" />
<p style="text-align: center;"><em>Gene Pool Size vs Loss</em></p>

<img src="images/fitness/Pool vs Div.png" width="750" style="display:block; margin:auto;" />
<p style="text-align: center;"><em>Gene Pool Size vs Diversity</em></p>

There are several interesting things to note about these figures. Focusing on the extremes first, only picking 10% of the best individuals is advantageous if we look purely at accuracy. However, this came at the cost of significantly reducing diversity, with such a small portion of the population passing through at each generation. Having too homogeneous a population can lead to getting stuck in local minima without exploring the wider loss landscape. On the other hand, having too many members of the population reproduce increases exploration of the loss landscape, but reduces selection pressure, as individuals with suboptimal solutions continue to reproduce. We can see this illustrated above as the accuracy lags far behind all of the other gene pool sizes. Keeping the best half performed is a strong middle ground with comparatively great accuracy, second only to keeping the top 10%, while remaining diverse.

We also investigated the effects of varying the probability that “sneakers”—individuals from the bottom 10% of the population—could enter the gene pool. We tested probabilities from 0–45%.

<img src="images/fitness/Snk vs Acc.png" width="750" style="display:block; margin:auto;" />
<p style="text-align: center;"><em>Sneaker Probability vs Accuracy</em></p>

<img src="images/fitness/Snk vs Loss.png" width="750" style="display:block; margin:auto;" />
<p style="text-align: center;"><em>Sneaker Probability vs Loss</em></p>

<img src="images/fitness/Snk vs Div.png" width="750" style="display:block; margin:auto;" />
<p style="text-align: center;"><em>Sneaker Probability vs Diversity</em></p>

Interestingly, across a range of sneaker probabilities, we didn't observe much variation in loss or diversity. So it doesn't impact our learning dynamics to a noticeable degree. However, having a 45% sneaker probability performed quite well, accuracy-wise. This may be a reflection of random variation of our dataset or starting genepool, but it may also suggest that a degree of genetic noise can occasionally help guide the population out of local minima. In future experiments, it would be insightful to set the hyperparameter to be above 50% and see the results.

Finally, we explored the impacts of elitism by varying the percentage of top-performing individuals who we carried unchanged to the next generation. 

<img src="images/fitness/Elite vs Acc.png" width="750" style="display:block; margin:auto;" />
<p style="text-align: center;"><em>Elitist Population Size vs Accuracy</em></p>

<img src="images/fitness/Elite vs Loss.png" width="750" style="display:block; margin:auto;" />
<p style="text-align: center;"><em>Elitist Population Size vs Loss</em></p>

<img src="images/fitness/Elite vs Div.png" width="750" style="display:block; margin:auto;" />
<p style="text-align: center;"><em>Elitist Population Size vs Diversity</em></p>


When we have too many elites, we slow down evolutionary convergence. We aren't introducing enough change from generation to generation to explore the landscape and improve our solution. We can see evidence of this in our stunted accuracy, low diversity, and higher loss when we increase the size of our elite population. However, when we eliminate elites or keep only 5%, we see noticeable improvements. Our loss is converging faster, we maintain a diverse population, and our accuracies after 500 generations are the highest. Keeping the elite population helps our accuracy, outperforming the population without elites by over 5% over 500 generations. Overall, we observed that on MNIST, modest elitism provides a valuable balance between preserving high-quality solutions and allowing diversity within the population.


### Inheritance and Parent Quantity:

##### Generated Data Experiment:

As a proof of concept, a multi parent classification experiment was run on generated 2 dimensional data with 0.2 noise and 300 points. The accuracy, loss, and euclidean diversity was tracked across 300 iterations. The experiment was run with the hyperparameter num_parents set to 0, 1, 2, 3, 5, and 10.

<img src="images/Multi-Parent/MP_GD_1_300.png" width="900" style="display:block; margin:auto;" />



<img src="images/Multi-Parent/MP_GD_2_300.png" width="450" style="display:block; margin:auto;" />
<p style="text-align: center;"><em>Figures demonstrating loss, diversity, and accuracy performance of Evolutionary Optimization on Generated data using 0, 1, 2, 3, 5, and 10 parents over 300 iterations</em></p>



As seen in the above visualization, Loss and Accuracy were comparable across all quantities of parents, while diversity varied significantly. In particular, with num_parents set to 0 and to a lesser extent 1, diversity was much lower than all other quantities of parents. The accuracy of the 0 parent model also lagged behind the other models over more iterations. Without functioning parents, evolution is replaced by random chance, as the heritability, defined as a requirement for evolution by @Lewontin1970units, is eliminated. 

While this had a much smaller impact on this relatively simple experiment of generated data, the implications on a much more complex classification problem, such as MNIST, could be significant. 

##### MNIST Experiment:

A similar experiment, performed on a subset 1000 images from the MNIST dataset tested the accuracy, loss, and diversity of num_parents = 0, 1, 2, 5, 10 over 1000 iterations.

<img src="images/Multi-Parent/MP_MNIST_1.png" width="900" style="display:block; margin:auto;" />


<img src="images/Multi-Parent/MP_MNIST_2.png" width="450" style="display:block; margin:auto;" />

<p style="text-align: center;"><em>Figures demonstrating loss, diversity, and accuracy performance of Evolutionary Optimization on a Subset of the MNIST dataset using 0, 1, 2, 5, and 10 parents over 1,000 iterations</em></p>

The benefits of inheritance are clear, as the zero parent model has a significantly higher loss and lower accuracy throughout the 1000 iterations compared to all other models. Additionally, the benefits of multi-parent inheritance are demonstrated by the declining improvement in accuracy when num_parents = 1. The lower diversity compared to the other models, and the importance of diversity in evolutionary algorithms in allowing for the exploration of the solution space, results in poorer performance of the single parent model. A single parent allows for the inheritance of a fitness,leading to better performance compared to the 0 parent model  @Lewontin1970units. However, it does not allow for large enough variation in fitness. With lower diversity, the 1 parent model is less likely to find a global minimum compared to the 2+ parent models. While it does find some form of a local minimum, the lack of diversity results in a drop off in improvement at around 600 iterations, while the models with 2, 5, and 10 parents continue to have spikes in improvement. 

In the context of classification of the MNIST dataset, evolutionary models benefit from the added diversity resulting from the use of larger quantities of parents contributing weights to each new child in the subsequent generation. While more computing power, and more iterations are required to truly optimize this hyperparameter, these experiments clearly demonstrate the benefits of multi-parent inheritance. 

### Hybridizing evolution with gradient descent:


### Adjustment from binary to multi-class classification:


### Mutation Methods:


### Final MNIST Results:

# Concluding Discussion:

# Group Contributions:

**Lukka:** As a unit, the whole team contributed to the conceptualization and the early stages of building a working prototype. I worked mainly on implementing, refining, and exploring the selection mechanisms in our evolutionary model. I also helped integrate the Laplacian mutation distribution. I also helped include and streamline my work and the work of others into a central working file. This was work that helped build the base of how we would handle our object-oriented programming approach and handle tuning hyperparameters. I also spent considerable effort and time getting MNIST to run correctly on the Middlebury cluster to facilitate larger-scale experimentation. In all the team meetings, we all spent time digging into one another's code, learning and helping on implementation, debugging, and developing conceptual frameworks.

# Personal Reflection: